In [20]:
import urllib.request
import re
from bs4 import BeautifulSoup
from lxml import html
import pandas as pd
# import xmltojson
import json
import requests

In [95]:
def range_price(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
    }
        
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "lxml")
    lowend = soup.find_all("span", {"class":"value", "itemprop":"lowprice", "content": True})
    highend = soup.find_all("span", {"class":"value", "itemprop":"highprice", "content": True})
    quantityList = []
    rangePrice = ""

    for i in lowend:
        rangePrice+=(i["content"])
        rangePrice+=" - "
        break

    for i in highend:
        rangePrice+=(i["content"])
        break

    return rangePrice


In [146]:
def scrape_handguns_helper(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
    }
    url+= "=undefined&start=0&sz=100"
        
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "lxml")
    ammo = soup.find_all("div", class_="pdp-link")
    ammoPrice = soup.find_all("div",  class_="tile-body")
    names = []
    prices = []
    for i in ammo:
        names.append(i.find("span").text)

    for i in ammoPrice:
        if '<span class="range">' in str(i):
            x = i.find("a", {"class": "link", "href":True})
            prices.append(x["href"])
        else:
            x = (i.find("span", class_="sales"))
            prices.append(x.find("span", {"class":"value", "content": True}))


    for i in prices:
        if str(type(i)) == "NoneType":
            continue

        elif "http" in i:
            x = range_price(i)
            prices.append(x)
            print(x)
        else:
            x = str(i)
            y = [float(s) for s in re.findall(r'-?\d+\.?\d*', x)]
            prices.append(str(y[0]))

            
            

    productList = pd.DataFrame(
        {"name": names, "price": prices}
    )
    print(productList)


    
    
    



    
    

In [147]:
# https://www.cheaperthandirt.com/9mm-luger-parabellum/ammunition/handgun-ammo/?https://www.cheaperthandirt.com/9mm-luger-parabellum/ammunition/handgun-ammo/=undefined&start=0&sz=36
handgunByCaliberLink = "https://www.cheaperthandirt.com/shop-by?cgid=78&searchBy=Caliber"
def scrape_handguns(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
    }
        
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "lxml")
    sections = soup.find_all("ul",class_= "col-lg-11 right-list")
    dirtyLinks = []  
    calibers = []  # names
    linkToCaliberList = []  # links
    
    for i in sections:
        dirtyLinks.append(i.find_all("a"))

    for i in dirtyLinks:
        for j in i:
            aLink = re.findall("(?P<url>https?://[^\s]+)", str(j))
            linkToCaliberList.append(str(aLink[0]).replace('">', ""))
            calibers.append(j.find("span").text)


    handgunCaliberLinks = pd.DataFrame(
        {"caliber": calibers, "links": linkToCaliberList}
    )

    productList = []
    for l in handgunCaliberLinks["links"]:
        productList.append(scrape_handguns_helper(l))
        
    return productList

In [148]:
scrape_handguns(handgunByCaliberLink)

TypeError: argument of type 'float' is not iterable

In [5]:
# aString = ["CTDhandgun", "CTDrifle", "CTDshotgun"]
# num = 0
# for i in AmmoList:
#     res = i.to_json(orient="split")
#     parsed = json.loads(res)
#     file = json.dumps(parsed, indent=2)
#     with open(f"{aString[num]}.json", "w") as fp:
#         json.dump(file, fp)
#     num += 1






# savePath = "/Users/chakaneshegog/Desktop/Ammo/ammo_finder/ammo_finder/jsonData/cheaperThanDirt"
# import os, shutil, glob
# try:
#     os.makedirs(savePath)  # creates a destination for directory
# except OSError:
#     pass

# for data in glob.iglob("*.json"):
#     shutil.copy2(data, savePath)